In [1]:
import sys
import logging
import os
import json
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(message)s")

# LOAD DATA

In [2]:
DATA_FOLDER = os.getcwd() + '/review_scorer/data/'
SENTI_PATH = DATA_FOLDER + 'SentiWord_info.json'
DATA_PATH = DATA_FOLDER + 'data_origin.csv'
data: pd.DataFrame

with open(SENTI_PATH, mode='rt', encoding='UTF8') as f:
    senti = pd.DataFrame.from_dict(json.load(f))

data = pd.read_csv(DATA_PATH, encoding='UTF8')
data = data.dropna(axis=0)
data = data.sample(frac=1).reset_index(drop=True)

# TOKENIZER

In [3]:
from tqdm import tqdm
from twkorean import TwitterKoreanProcessor

processor = TwitterKoreanProcessor()
tokenize = processor.tokenize_to_strings
tokens = [tokenize(_) for _ in tqdm(data.review)]

100%|████████████████████████████████████| 71904/71904 [09:06<00:00, 131.67it/s]


# SET REVIEW SCORER

In [4]:
from review_scorer import Doc2Category

# Review scorer needs tokens of datas to train when initializing it.
# 리뷰 채점기 클래스를 생성할 때, 토크나이즈 된 데이터를 인자로 주어야 합니다.
rs = Doc2Category(sentences=tokens, senti_dict_path=SENTI_PATH)

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 1104467 words, keeping 28336 word types
PROGRESS: at sentence #20000, processed 2207760 words, keeping 37206 word types
PROGRESS: at sentence #30000, processed 3281348 words, keeping 43142 word types
PROGRESS: at sentence #40000, processed 4382669 words, keeping 47162 word types
PROGRESS: at sentence #50000, processed 5453780 words, keeping 49475 word types
PROGRESS: at sentence #60000, processed 6524294 words, keeping 51097 word types
PROGRESS: at sentence #70000, processed 7625145 words, keeping 51655 word types
collected 51667 word types from a corpus of 7833944 raw words and 71904 sentences
Creating a fresh vocabulary
Doc2Category lifecycle event {'msg': 'effective_min_count=5 retains 22483 unique words (43.52% of original 51667, drops 29184)', 'datetime': '2022-08-12T02:56:33.737712', 'gensim': '4.2.0', 'python': '3.7.12 | packaged by cond

## TAGGING RIVIEW SCORER's SENTIMENTAL DICTIONARY

In [9]:
# Tagging review scorer's sentimental dictionary by category.
# 카테고리에 따라 리뷰 채점기의 감성사전을 태깅합니다.
rs.tag(categories={'taste': ['맛', '맛있다', '맛없다'],
                   'price': ['가격', '싸다', '비싸다', '저렴'],
                   'service': ['서비스', '친절'],
                   'atmosphere': ['인테리어', '분위기']},
       width=4, depth=4)

category "taste" tagged
category "price" tagged
category "service" tagged
category "atmosphere" tagged


# SCORING WITH REVIEW SCORER

In [24]:
start = 4000
for i in range(start, start + 10):
    print('index: ', i)
    print(rs.score_review(tokenize(data.review.iloc[i])))
    print(data.review.iloc[i])
    print()

index:  4000
{'taste': 5, 'price': -2, 'service': 0, 'atmosphere': 0}
딤섬이 맛있다 하여 고고. 웬걸, 부추와 새우가 가득 들어있는 부채교는 맛있었으나 샤오롱바오는 육즙도 밍밍하고 먹기도 불편하고 피도 별로.혹시나 해서 마라샤오롱바오 도 추가했는데 마라향은 좋았으나 역시 먹기 불편하고 그냥 그랬음. 의외로 탄탄면이 고쏘하고 칼칼해서 딤섬과 탄탄면을 교차로 흡입. 국물까지 싹싹 흡입. 전체적으로는 맛있다까지는 아니고 괜찮은편

index:  4001
{'taste': 2, 'price': 0, 'service': 0, 'atmosphere': 0}
오마카세를 즐기는 이라면 누구든 한번쯤 가봤거나, 가고자 하는 스시야. 부푼 기대를 안고 방문한 스시조 런치. 섬세한 접객, 최상의 재료!그냥 맛있는 스시.이 재료에 이 맛은 당연한데…?런치기준 10~12만원 정도면“아주 맛있다!”겠지만,최고의 가격대에 걸맞는 극강의 맛과 분위기는 아니어서 조금 아쉽다.

index:  4002
{'taste': 2, 'price': 0, 'service': 0, 'atmosphere': 0}
해수염 맛있다! 브룬디 coe 드립에 버터롤빵 먹으니 참으로 맛있군

index:  4003
{'taste': 1, 'price': 0, 'service': 0, 'atmosphere': 0}
부산에서 거의 유일하게 먹은 한 끼. 평점이 굉장히 높은데 부산이라 높은 느낌..?라멘 종류가 두 가지인것 같았는데 저희가 갔을 땐 한 개가 품절이어서 한 메뉴만 가능했어요. 하나는 교자 세트로 시키고 하나는 라멘 단품으로 시켰어요. 좁디 좁은 라멘집만 보다가 이렇게 넓은 라멘집은 처음 봐서 놀랐어요. 엄청 넓어요 주방도 넓고 수용인원도 꽤 되는 느낌! 그래서 회전율이 빠르더라구요.라멘은 엄청 진했어요. 면은 얇고 툭툭 끊기는 면. 교자는 그냥 괜찮았어요. 간장을 엄청 뿌려주신거 같았는데 간장 맛은 거의 안 났다는 놀라운 진실,,그래서 저한텐 좀 밍밍

# PRINT TAGGED WORDS

In [11]:
rs.print_trees()

taste
├── 맛
│   ├── 감칠맛
│   │   ├── 단맛
│   │   │   ├── 신맛
│   │   │   ├── 짠맛
│   │   │   ├── 감칠맛
│   │   │   └── 새콤함
│   │   ├── 불향
│   │   │   ├── 불맛
│   │   │   ├── 감칠맛
│   │   │   ├── 향
│   │   │   └── 단맛
│   │   ├── 맛
│   │   │   ├── 감칠맛
│   │   │   ├── 단맛
│   │   │   ├── 느낌
│   │   │   └── 향
│   │   └── 육향
│   │       ├── 감칠맛
│   │       ├── 짠맛
│   │       ├── 향
│   │       └── 단맛
│   ├── 단맛
│   │   ├── 신맛
│   │   │   ├── 짠맛
│   │   │   ├── 단맛
│   │   │   ├── 새콤함
│   │   │   └── 산미
│   │   ├── 짠맛
│   │   │   ├── 신맛
│   │   │   ├── 단맛
│   │   │   ├── 새콤함
│   │   │   └── 끝맛
│   │   ├── 감칠맛
│   │   │   ├── 단맛
│   │   │   ├── 불향
│   │   │   ├── 맛
│   │   │   └── 육향
│   │   └── 새콤함
│   │       ├── 단맛
│   │       ├── 신맛
│   │       ├── 짠맛
│   │       └── 짭쪼름함
│   ├── 느낌
│   │   ├── 맛
│   │   │   ├── 감칠맛
│   │   │   ├── 단맛
│   │   │   ├── 느낌
│   │   │   └── 향
│   │   ├── 스타일
│   │   │   ├── 타입
│   │   │   ├── 취향
│   │   │   ├── 스탈
│   │   │   └── 느낌
│   │   ├── 타입
│   │   │   ├── 스타일
│  

In [23]:
for root in rs.word_roots:
    words_set = list(map(lambda x: x.data, root.descendants))
    print(f'{root.name}({len(words_set)}개)')
    print(*words_set[:10], "..")

taste(255개)
맛 감칠맛 단맛 신맛 짠맛 감칠맛 새콤함 불향 불맛 감칠맛 ..
price(340개)
가격 금액 비용 요금 금액 현금 사항 가격 금액 값 ..
service(170개)
서비스 친절 접객 친절 응대 매너 서비스 청결 시설 친절 ..
atmosphere(170개)
인테리어 분위기 인테리어 분위기 내부 음악 무드 내부 실내 인테리어 ..
